### 豆瓣评分的预测

在这个项目中，我们要预测一部电影的评分，这个问题实际上就是一个分类问题。给定的输入为一段文本，输出为具体的评分。 在这个项目中，我们需要做：
- 文本的预处理，如停用词的过滤，低频词的过滤，特殊符号的过滤等
- 文本转化成向量，将使用三种方式，分别为tf-idf, word2vec以及BERT向量。 
- 训练逻辑回归和朴素贝叶斯模型，并做交叉验证
- 评估模型的准确率

在具体标记为``TODO``的部分填写相应的代码。 

In [18]:
#导入数据处理的基础包
import numpy as np
import pandas as pd

#导入用于计数的包
from collections import Counter

#导入tf-idf相关的包
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer

#导入模型评估的包
from sklearn import metrics

#导入与word2vec相关的包
from gensim.models import KeyedVectors

#导入与bert embedding相关的包，关于mxnet包下载的注意事项参考实验手册
from bert_embedding import BertEmbedding
import mxnet

#包tqdm是用来对可迭代对象执行时生成一个进度条用以监视程序运行过程
from tqdm import tqdm

#导入其他一些功能包
import requests
import os

### 1. 读取数据并做文本的处理
你需要完成以下几步操作：
- 去掉无用的字符如！&，可自行定义
- 中文分词
- 去掉低频词

In [19]:
#读取数据
data = pd.read_csv('data/DMSC.csv')
#观察数据格式
data.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,10,Avengers Age of Ultron,复仇者联盟2,2017-01-22,11,影志,2015-04-30,4,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,381
2,20,Avengers Age of Ultron,复仇者联盟2,2017-01-22,21,随时流感,2015-04-28,2,奥创弱爆了弱爆了弱爆了啊！！！！！！,120
3,30,Avengers Age of Ultron,复仇者联盟2,2017-01-22,31,乌鸦火堂,2015-05-08,4,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,30
4,40,Avengers Age of Ultron,复仇者联盟2,2017-01-22,41,办公室甜心,2015-05-10,5,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,16


In [20]:
#输出数据的一些相关信息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212506 entries, 0 to 212505
Data columns (total 10 columns):
ID               212506 non-null int64
Movie_Name_EN    212506 non-null object
Movie_Name_CN    212506 non-null object
Crawl_Date       212506 non-null object
Number           212506 non-null int64
Username         212496 non-null object
Date             212506 non-null object
Star             212506 non-null int64
Comment          212506 non-null object
Like             212506 non-null int64
dtypes: int64(4), object(6)
memory usage: 16.2+ MB


In [21]:
#只保留数据中我们需要的两列：Comment列和Star列
data = data[['Comment','Star']]
#观察新的数据的格式
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,3
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,4
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,2
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,4
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,5


In [22]:
# 这里的star代表具体的评分。但在这个项目中，我们要预测的是正面还是负面。我们把评分为1和2的看作是负面，把评分为3，4，5的作为正面
data['Star']=(data.Star/3).astype(int)
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1


#### 任务1： 去掉一些无用的字符

In [ ]:
# TODO1: 去掉一些无用的字符，自行定一个字符几何，并从文本中去掉
#    your to do 


#### 任务2：使用结巴分词对文本做分词

In [12]:
# TODO2: 导入中文分词包jieba, 并用jieba对原始文本做分词
import jieba
def comment_cut(content):
    # TODO: 使用结巴完成对每一个comment的分词
    pass

# 输出进度条
tqdm.pandas(desc='apply')
data['comment_processed'] = data['Comment'].progress_apply(comment_cut)

apply: 100%|██████████| 212506/212506 [00:00<00:00, 1005813.56it/s]


In [13]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国。,0,None
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,0,None
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,None
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,0,None
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,0,None


#### 任务3：设定停用词并去掉停用词

In [14]:
# TODO3: 设定停用词并从文本中去掉停用词

# 下载中文停用词表至data/stopWord.json中，下载地址:https://github.com/goto456/stopwords/
if not os.path.exists('data/stopWord.json'):
    stopWord = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt")
    with open("data/stopWord.json", "wb") as f:
         f.write(stopWord.content)

# 读取下载的停用词表，并保存在列表中
with open("data/stopWord.json","r") as f:
    stopWords = f.read().split("\n")  
    
    
# 去除停用词
def rm_stop_word(wordList):
    # your code, remove stop words
    # TODO

#这行代码中.progress_apply()函数的作用等同于.apply()函数的作用，只是写成.progress_apply()函数才能被tqdm包监控从而输出进度条。
data['comment_processed'] = data['comment_processed'].progress_apply(rm_stop_word)

IndentationError: expected an indented block (<ipython-input-14-bb189d611c89>, line 20)

In [ ]:
# 观察新的数据的格式
data.head()

#### 任务4：去掉低频词，出现次数少于10次的词去掉

In [ ]:
# TODO4: 去除低频词, 去掉词频小于10的单词，并把结果存放在data['comment_processed']里


data['comment_processed'] = 

### 2. 把文本分为训练集和测试集
选择语料库中的20%作为测试数据，剩下的作为训练数据

In [ ]:
# TODO5: 把数据分为训练集和测试集. comments_train（list)保存用于训练的文本，comments_test(list)保存用于测试的文本。 y_train, y_test是对应的标签（0、1）

test_ratio = 0.2
comments_train, comments_test = 
y_train, y_test = 

### 3. 把文本转换成向量的形式

在这个部分我们会采用三种不同的方式:
- 使用tf-idf向量
- 使用word2vec
- 使用bert向量

转换成向量之后，我们接着做模型的训练

#### 任务6：把文本转换成tf-idf向量

In [15]:
# TODO6: 把训练文本和测试文本转换成tf-idf向量。使用sklearn的feature_extraction.text.TfidfTransformer模块
#    请留意fit_transform和transform之间的区别。 常见的错误是在训练集和测试集上都使用 fit_transform，需要避免！ 
#    另外，可以留意一下结果是否为稀疏矩阵

tfidf_train=
tfidf_test=
print (tfidf_train.shape, tfidf_test.shape)

SyntaxError: invalid syntax (<ipython-input-15-a3db87312644>, line 5)

#### 任务7：把文本转换成word2vec向量

In [ ]:
# 由于训练出一个高效的word2vec词向量往往需要非常大的语料库与计算资源，所以我们通常不自己训练Wordvec词向量，而直接使用网上开源的已训练好的词向量。
# data/sgns.zhihu.word是从https://github.com/Embedding/Chinese-Word-Vectors下载到的预训练好的中文词向量文件
# 使用KeyedVectors.load_word2vec_format()函数加载预训练好的词向量文件
model = KeyedVectors.load_word2vec_format('data/sgns.zhihu.word')

In [ ]:
#预训练词向量使用举例
model['今天']

In [ ]:
# TODO7: 对于每个句子，生成句子的向量。具体的做法是：包含在句子中的所有单词的向量做平均。
vocabulary = model.vocab

word2vec_train=
word2vec_test=
print (word2vec_train.shape, word2vec_test.shape)

#### 任务8：把文本转换成bert向量

In [16]:
# 导入gpu版本的bert embedding预训练的模型。
# 若没有gpu，则ctx可使用其默认值cpu(0)。但使用cpu会使程序运行的时间变得非常慢
# 若之前没有下载过bert embedding预训练的模型，执行此句时会花费一些时间来下载预训练的模型
ctx = mxnet.gpu()
embedding = BertEmbedding(ctx=ctx)

# TODO8: 跟word2vec一样，计算出训练文本和测试文本的向量，仍然采用单词向量的平均。
bert_train=
bert_test=
print (bert_train.shape, bert_test.shape)

SyntaxError: invalid syntax (<ipython-input-16-42afd78e0bdb>, line 8)

In [ ]:
print (tfidf_train.shape, tfidf_test.shape)
print (word2vec_train.shape, word2vec_test.shape)
print (bert_train.shape, bert_test.shape)

### 4. 训练模型以及评估
对如上三种不同的向量表示法，分别训练逻辑回归模型，需要做：
- 搭建模型
- 训练模型（并做交叉验证）
- 输出最好的结果

In [ ]:
# 导入逻辑回归的包
from sklearn.linear_model import LogisticRegression

#### 任务9：使用tf-idf，并结合逻辑回归训练模型

In [ ]:
# TODO9: 使用tf-idf + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数



print('TF-IDF LR test accuracy %s' % metrics.accuracy_score(y_test, tf_idf_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('TF-IDF LR test F1_score %s' % metrics.f1_score(y_test, tf_idf_y_pred,average="macro"))

#### 任务10：使用word2vec，并结合逻辑回归训练模型

In [ ]:
# TODO10: 使用word2vec + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数



print('Word2vec LR test accuracy %s' % metrics.accuracy_score(y_test, word2vec_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Word2vec LR test F1_score %s' % metrics.f1_score(y_test, word2vec_y_pred,average="macro"))

#### 任务11：使用bert，并结合逻辑回归训练模型

In [ ]:
# TODO11: 使用bert + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数



print('Bert LR test accuracy %s' % metrics.accuracy_score(y_test, bert_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Bert LR test F1_score %s' % metrics.f1_score(y_test, bert_y_pred,average="macro"))

#### 任务12：对于以上结果请做一下简单的总结，按照1，2，3，4提取几个关键点，包括：
- 结果说明什么问题？
- 接下来如何提高？

1.
2.
3.
4.
5.
6.